# **Actividad 03**
----
> **Alumna :** Claudia Luz Rojas Soto \
> **Asignatura:** Minería de Datos                                        
> **Docente:** Carlos Fernando Montoya Cubas

## **Instalación de Librerías**
---

In [1]:
import numpy as np  
import pandas as pd  
import itertools

## **Módulos a necesitar**
---

***findsubsets(C,n):*** Recibe como parámetros C que es conjunto de datos de los cuales queremos generar combinaciones de sus elementos y n que es la cantidad de elementos de cada combinación.

In [2]:
# Importamos funciones de la libreria itertools que permiten realizar las combinaciones
from itertools import chain, combinations
def findsubsets(C,n): 
    # Devolvemos un conjunto de combinaciones de n elementos
    return set(itertools.combinations(C,n))

***print_rules(rules):*** Recibe como parámetro un arreglo de reglas y las imprime con un formato preestablecido mostrando el soporte, la confianza y el sustento(lift).

In [180]:
def print_rules(rules):
    # Dentro de cada arreglo de reglas encontraremos una regla con la estructura:
    # [ index , antecedente, consecuente, support, confidence, lift]
    for rule in rules:
        # Mostrando estos valores con la siguiente estructura
        print(" ======= REGLA "+ str(rule.index) + " =======")
        if(len(rule[0])==1):
            print(rule[0],'--------',rule[1])
        else:
            print(rule[0],'--------',rule[1])
        print('SUPPORT    = ',rule[2])
        print('CONFIDENCE = ',rule[3])
        print('LIFT       = ',rule[4])

## **Datos**
---

### **Lectura de Datos**
---
***spotify.npy:*** Este conjunto de datos contiene distintas playlists compuestas por distintas canciones. Leemos este conjunto de datos volviendolo una lista de lista.

In [4]:
data = np.load('spotify.npy', allow_pickle=True)
data = data.tolist()
# Convertimos la data a una lista de listas
playlists = [data[i] for i in data ]

***Tamaño del dataset:***

In [5]:
len(playlists)

10000

***Primer item:***


In [6]:
# Mostramos un playlist del conjunto de datos
print(playlists[0:1])
# Mostramos el tamaño de este playlist
print('\n El tamaño de la lista es:',len(playlists[0:1][0]))

[['Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Right Where You Want Me - Radio Edit Version', 'Beautiful Soul', "Leavin'", 'Me & U', 'Ice Box', 'Sk8er Boi', 'Run It!', 'Check On It - feat. Bun B and Slim Thug', "Jumpin', Jumpin'", 'Soak Up The Sun', 'Where Is The Love?', "Stacy's Mom", 'Just The Girl', 'Yo (Excuse Me Miss)', 'Year 3000', 'Lip Gloss', 'Everytime We Touch - Radio Edit', 'Whatcha Say', 'Miss Independent', 'Party In The U.S.A.', 'The Great Escape', 'Replay', 'Forever', 'Your Love Is My Drug', 'Closer', 'One Less Lonely Girl', 'Paper Planes', 'Mr. Brightside', 'All The Small Things', 'Beep', 'Somebody To Love', 'Dirty Little Secret', 'Baby', 'A Thousand Miles', 'Livin on Sunday', 'See You Again', 'How Do You Sleep? - Featuring Ludacris', 'This Is Me', 'My Happy Ending', 'Check Yes Juliet', 'The Great Escape']]

 El tamañ

### **Declaración de parametros**
---
- ***veces_minimas_aparece_en_lista:***  100
- ***soporte_min:*** veces_minimas_aparece_en_lista / cantidad de playlists
- ***confianza_min:*** 0.5
- ***sustento_min:*** 1.5

In [7]:
veces_minimas_aparece_en_lista=100
soporte_min = veces_minimas_aparece_en_lista / len(playlists)
confianza_min =0.5
sustento_min =1.5

print("Soporte:",soporte_min)

Soporte: 0.01


## **Implementar el algoritmo Apriori**
---

### **Módulo get_frequent_itemsets**
---
#### Input: 
- *playlists:* Arreglo de arreglos, donde cada uno de estos arreglos representa un conjunto de canciones.
- *min_support:* Umbral de soporte.

#### Output:
- *count_item:* Diccionario con itemsets frecuentes, y sus valores support, para cada uno de estos.


In [8]:
def get_frequent_itemsets(playlists, min_support):
    # Diccionario que guardara nuestros itemsets frecuentes y su soporte
    count_item= dict()
    # Base: 1-itemset
    # Recorremos cada playlist en el conjunto de playlist
    for playlist in playlists:
        # Recorremos cada canción del playlist
        for song in playlist:
            # Si la cancion no esta en nuestro diccionario, lo agregamos y empezamos a recuperar el numero de apariciones en 1
            if (not song in count_item):
                count_item[song] = 1
            # Si la canción ya está en nuestro diccionario, incrementamos el número de apariciones de este en uno
            else:
                count_item[song] = count_item[song] + 1
    # Luego de obtener el numero de apariciones de cada item de nuestro diccionario
    # Calculamos el soporte de estos, y validamos si pasa el min_support
    # En caso no pase el umbral, sacamos el item del diccionario
    for item in list(count_item): 
        count_item[item] = count_item[item]/len(playlists)
        if(count_item[item] < min_support):
            count_item.pop(item)
    
    # Obtención de k-itemsets
    k = 2 # Iniciamos en 2-itemsets
    pos = 0 
    bandera = True # Bandera que nos indicara si ya se obtuvieron todos los itemsets frecuentes
    # Si la bandera es false termina el bucle
    while(bandera):
        # Recuperamos los keys del diccionario en una lista
        lista = list(count_item)
        # Nueva lista : Lista auxiliar que guardará los posibles nuevos itemsets frecuentes
        nueva_lista = []
        # Recorremos los items que existen en la lista y los emparejamos con el resto de su forma (de manera similar a una matriz)
        # Para esto será necerario guardar la posición de la ultimo k itemset obtenido
        # Este pos nos permitira recorrer sectores de la lista
        # Por ejemplo, en el caso de la forma 2-itemset frec., el recorrido de la lista empezaria cuando acaben los 1-itemsets frec. 
        # Y así sucesivamente
        for i in range(pos,len(lista)):
            for j in range(i+1,len(lista)):
                # Para obtener posibles itemsets tenemos dos casos, que sea 2-itemsets o k-itemsets.
                if(k==2):
                    # En el caso de que estemos generando 2-itemsets, mezclamos todos los 1-itemsets 
                    nuevo_item = set()
                    nuevo_item.add(lista[i])
                    nuevo_item.add(lista[j]) 
                    nuevo_item = list(nuevo_item)
                    if(len(nuevo_item)==k):
                        # Agregamos el nuevo item ordenado en la lista auxiliar
                        nueva_lista.append(tuple(sorted(nuevo_item)))
                else:
                    # En el caso de los k-itemsets
                    # Un candidato es la mezcla de los k-1 itemsets que tengan los mismos k-2 primeros items
                    if(lista[i][:k-2]==lista[j][:k-2]):
                        # En caso se cumpla esto, realizamos las union de los k-1 itemsets, y se agrega el candidato (ordenado) a la lista auxiliar
                        nuevo=list(set(lista[i]).union(set(lista[j])))
                        if not nuevo in nueva_lista:
                            nueva_lista.append(tuple(sorted(nuevo)))
        
        # Recuperamos la posición donde se encuentra el primer k-1 itemset
        pos = len(lista)

        # Si la lista auxiliar tiene candidatos a itemsets frecuentes
        # Los añadimos al diccionario inicialmente creado
        if(len(nueva_lista)!=0):
            # Por cada item candidato en la lista auxiliar
            for item_nuevo in nueva_lista:
                # ----- FALTA VALIDACIÓN
                # Buscamos si es parte de algun playlist
                for playlist in playlists: 
                    # Realizamos esto con la intersección de conjuntos
                    if(set(item_nuevo).intersection(playlist) == set(item_nuevo)):
                        # Si el item set candidato se encuentra en algun playlist lo agregamos al diccionario con un value de 1.
                        if (not item_nuevo in count_item.keys()):
                            count_item[item_nuevo] = 1
                        # Caso contrario, incrementamos el value del itemset candidato
                        else:
                            count_item[item_nuevo] = count_item[item_nuevo] + 1 
                # Obtenemos el support del itemset agregado (si este ha sido agregado)
                if (item_nuevo in count_item.keys()):
                    count_item[item_nuevo] = count_item[item_nuevo]/len(playlists)
                    # En caso no pase el umbral de min_support lo eliminamos del diccionario 
                    if count_item[item_nuevo] < min_support:
                        count_item.pop(item_nuevo)
        #Caso contrario, volvemos la bandera a false, ya que no se podran obtener más itemsets frecuentes.
        else:
            bandera=False
        # Agregamos 1 al tamaño del itemset que se quiere generar
        k = k+1
        # Generamos...
    # Guardamos los valores del diccionario de la forma [itemset_frecuente, support] en un archivo .csv
    df=pd.DataFrame.from_dict(count_item,orient="index")
    df.to_csv('./playlists-'+str(len(playlists))+'-'+str(min_support)+'.csv')
    return count_item

### **Módulo generate_association_rules**
---
Input: 
- *frequent_itemsets:* Diccionario de la forma donde cada elemento es de la forma: **(itemset frecuente: support)**.
- *confidence:* Umbral de confianza, default = 0.
- *lift:* Umbral de lift, default = 0.

Output:
- *rules:* Arreglo de reglas que guardan el antecedente, consecuente, support, confidence y lift de cada una de estas.

In [181]:
from ast import literal_eval

def generate_association_rules(frequent_itemsets, confidence = 0, lift = 0):   
    # Single_tam es la posicion donde esta el ultimo 1-itemset
    single_tam = 0
    # Para obtener este valor recorremos las keys del diccionario
    items = list(frequent_itemsets)
    values = list(frequent_itemsets.values())
    # Existe un problema al leer el diccionario desde un .csv ya que no lee tuplas si no strings
    # La siguiente parte de codigo crea un diccionario auxiliar que nos permitira eliminar este error
    aux_dict = dict()
    for i in range(len(items)):
        val = items[i]
        if val.startswith("("):
            val = literal_eval(val)
            items[i] = val
            aux_dict[val] = values[i]
        else:
            single_tam = single_tam + 1
            aux_dict[items[i]] = values[i]
    # Recuperamos las keys del diccionario de itemsets frecuentes
    count = 0
    rules = []
    # Recorremos este arreglo de atras para adelante, es decir, desde los itemsets con k elementos hasta los itemsets con 2 elementos, ayudandonos del single_tam anterior.
    for i in range(len(items)-1,single_tam,-1):
        # Recuperamos el tamaño del item
        tam_item = len(items[i])
        # Para generar las reglas empezamos a partir de un tamaño de tam_item - 1
        tam_antecedente = tam_item - 1
        # Mientras el tamaño del antecedente sea mayor que uno buscamos reglas
        while tam_antecedente>=1:
            # Generamos combinaciones a partir de los elementos de itemset frecuente con el tamaño del antecedente, predefinido anteriormente
            subconjuntos = list(findsubsets(items[i],tam_antecedente))
            # Recogemos el valor del support del item analizado
            x_u_y = aux_dict[items[i]]
            # Para cada combinación con k (tam_antecedente) items
            for elemento in subconjuntos:
                # -FALTA PODA CONFIDENCE 
                # Recogemos el valor del support del antecedente      
                if (len(elemento) == 1): 
                    x = aux_dict[str(elemento[0])] # Elemento[0] lo realizamos para recogerlo como string ya que solo es 1 elemento.
                else:
                    x = aux_dict[elemento]
                # Obtenemos el consecuente
                item_diff = tuple(sorted(list(set(items[i]).difference(elemento))))
                # Si es solo un valor, lo recogemos como string para buscarlos
                if(len(item_diff)==1):
                    item_diff = list(item_diff)[0]
                
                # Obtenemos el valor del confidence dado por:
                # c( x -> y ) = sup(x u y) / sup ( x )
                confidence_ob = x_u_y/x
                # l(x -> y) = confidence / sup (y)
                # sup (y) = frequent_itemsets[item_diff]
                lift_ob = confidence_ob/aux_dict[item_diff]
                # Si los valores obtenidos pasan el umbral, agregamos la regla a nuestro arreglo de reglas con un index.
                if (confidence_ob>= confidence and lift_ob>= lift):
                    count = count + 1
                    # Si el elemento agregado es solo uno lo volvemos string
                    if(len(elemento)==1):
                        rules.append([elemento[0],item_diff,x_u_y,confidence_ob,lift_ob])
                    # Caso contrario continua como tupla
                    else:
                        rules.append([elemento,item_diff,x_u_y,confidence_ob,lift_ob])
            # Reducimos el tamaño del antecedente, para generar las combinaciones con este tamaño
            tam_antecedente = tam_antecedente - 1
    return rules



## **Aplicar el algoritmo y obtener reglas de asociación**
---

In [ ]:
# 10000 playlists
get_frequent_itemsets(playlists,0.01)
get_frequent_itemsets(playlists,0.02)
# 5000 playlists
get_frequent_itemsets(playlists[:5000],0.01)
get_frequent_itemsets(playlists[:5000],0.02)
# 2500 playlists
get_frequent_itemsets(playlists[:2500],0.01)
get_frequent_itemsets(playlists[:2500],0.02)
# 1250 playlists
get_frequent_itemsets(playlists[:1250],0.01)
get_frequent_itemsets(playlists[:1250],0.02)

Para la obtención de los itemsets frecuentes se hizo distintos experimentos con el dataset. Dentro de los cuales tenemos los siguientes:
 - playlists-10000-0.01.csv
 - playlists-10000-0.02.csv
 - playlists-5000-0.01.csv
 - playlists-5000-0.02.csv
 - playlists-2500-0.01.csv
 - playlists-2500-0.02.csv
 - playlists-1250-0.01.csv
 - playlists-1250-0.02.csv

Siendo el valor despues del string playlist, el tamaño del dataset tomado, y el valor siguiente el min-support con el que fue evaluado este dataset.


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-10000-0.01.csv', los cuales fueron obtenidos de las 5000 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [183]:
frequent_itemsets_10000_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-10000-0.01.csv?token=AOMD44CGNK7WJSOIX5RVWXTBUQTYU', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_10000_01.pop(list(frequent_itemsets_10000_01.keys())[0])
rules_10000_01 = generate_association_rules(frequent_itemsets_10000_01,0.5,1.2)
rules_pd_10000_01 = pd.DataFrame(rules_10000_01,columns = ["Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_10000_01= rules_pd_10000_01.sort_values('Lift',ascending=False)
rules_pd_10000_01.to_csv("rules_10000_01.csv")
rules_pd_10000_01.head(10)

,Antecedente,Consecuente,Support,Confidence,Lift
92,X (feat. Future),No Heart,0.0103,0.502439,33.720740
91,No Heart,X (feat. Future),0.0103,0.691275,33.720740
105,Knee Deep (feat. Jimmy Buffett),Chicken Fried,0.0107,0.656442,30.112005
107,You Was Right,Money Longer,0.0117,0.587940,29.844655
106,Money Longer,You Was Right,0.0117,0.593909,29.844655
56,Butterfly Effect,Bank Account,0.0111,0.572165,24.769045
83,Magnolia,Bank Account,0.0113,0.565000,24.458874
7,"(HUMBLE., Tunnel Vision)",XO TOUR Llif3,0.0102,0.733813,22.236756
31,"(Bounce Back, Broccoli (feat. Lil Yachty))",Bad and Boujee (feat. Lil Uzi Vert),0.0100,0.775194,21.960164
48,"(HUMBLE., XO TOUR Llif3)",DNA.,0.0102,0.500000,21.551724


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-5000-0.01.csv', los cuales fueron obtenidos de las 5000 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [184]:
frequent_itemsets_5000_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-5000-0.01.csv?token=AOMD44D5BJMWDNNBYM56ZZLBUQTV4', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_5000_01.pop(list(frequent_itemsets_5000_01.keys())[0])
rules_5000_01 = generate_association_rules(frequent_itemsets_5000_01,0.5,1.2)
rules_pd_5000_01 = pd.DataFrame(rules_5000_01,columns = ["Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_5000_01 = rules_pd_5000_01.sort_values('Lift',ascending=False)
rules_pd_5000_01.to_csv("rules_5000_01.csv",header=None)
rules_pd_5000_01.head(10)

,Antecedente,Consecuente,Support,Confidence,Lift
130,Knee Deep (feat. Jimmy Buffett),Toes,0.0102,0.593023,34.478096
129,Toes,Knee Deep (feat. Jimmy Buffett),0.0102,0.593023,34.478096
145,You Was Right,Money Longer,0.0110,0.604396,32.148702
144,Money Longer,You Was Right,0.0110,0.585106,32.148702
139,Play It Again,That's My Kind Of Night,0.0100,0.520833,31.001984
140,That's My Kind Of Night,Play It Again,0.0100,0.595238,31.001984
60,"(Bank Account, XO TOUR Llif3)",Magnolia,0.0102,0.680000,30.088496
132,Santeria,What I Got,0.0108,0.504673,30.040053
133,What I Got,Santeria,0.0108,0.642857,30.040053
61,"(Magnolia, XO TOUR Llif3)",Bank Account,0.0102,0.728571,28.683915


Para producir las reglas desde nuestros itemsets frecuentes de 'playlists-2500-0.01.csv', los cuales fueron obtenidos de las 2500 primeras canciones con un support de 0.01. Ejecutamos la función generate_association_rules con un confidence de 0.5 y un lift de 1.2.

In [185]:
frequent_itemsets_2500_01 = pd.read_csv('https://raw.githubusercontent.com/Claudiars20/BIGDATA2021/main/ACTIVIDAD03/experiments/playlists-2500-0.01.csv?token=AOMD44H5U2KAWPTL7G6PH4DBUQQQS', header=None, index_col=0, squeeze=True).to_dict()
frequent_itemsets_2500_01.pop(list(frequent_itemsets_2500_01.keys())[0])
rules_2500_01= generate_association_rules(frequent_itemsets_2500_01,0.5,1.2)
rules_pd_2500_01 = pd.DataFrame(rules_2500_01,columns = ["Antecedente", "Consecuente", "Support","Confidence","Lift"])
rules_pd_2500_01= rules_pd_2500_01.sort_values('Lift',ascending=False)
rules_pd_2500_01.to_csv("rules_2500_01.csv")
rules_pd_2500_01.head(10)


,Antecedente,Consecuente,Support,Confidence,Lift
171,Toes,Knee Deep (feat. Jimmy Buffett),0.0104,0.650000,38.690476
172,Knee Deep (feat. Jimmy Buffett),Toes,0.0104,0.619048,38.690476
29,"(Mask Off, iSpy (feat. Lil Yachty))",Rolex,0.0100,0.609756,35.450936
31,Rolex,"(Mask Off, iSpy (feat. Lil Yachty))",0.0100,0.581395,35.450936
170,What I Got,Santeria,0.0100,0.675676,34.473249
169,Santeria,What I Got,0.0100,0.510204,34.473249
109,That's My Kind Of Night,Play It Again,0.0104,0.702703,33.783784
108,Play It Again,That's My Kind Of Night,0.0104,0.500000,33.783784
18,"(Bank Account, rockstar)",Butterfly Effect,0.0100,0.781250,33.674569
16,"(Bank Account, Butterfly Effect)",rockstar,0.0100,0.781250,33.674569


## **Explicar las reglas obtenidas**
---